In [1]:
#Importing of necessary packages
import nltk
import re
import emoji
import demoji
import torch

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from datetime import datetime

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.model_selection import train_test_split

from transformers import BertTokenizer, BertModel, pipeline
from numpy import loadtxt, savetxt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from xgboost import XGBClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Conv1D, MaxPool1D, BatchNormalization, Bidirectional
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score

#import bayes opt for hyperparameter tuning
from bayes_opt import BayesianOptimization

In [2]:
# Reading in the dataset
df_human = pd.read_csv('human_tweets_processed.csv')
df_fake = pd.read_csv('bot_tweets_fake_processed.csv')
df_social = pd.read_csv('bot_tweets_social_processed.csv')

In [3]:
df_tweets_all = pd.concat([df_human, df_fake, df_social])
df_tweets_all = df_tweets_all.reset_index(inplace = False, drop = True)

In [4]:
df_tweets = df_tweets_all[["text", "cleaned_text", "bot"]]
df_tweets = df_tweets.rename(columns = {"bot": "isBot"})
df_tweets

,text,cleaned_text,isBot
0,@KyleDavidHall @YouTube YASS THANKS BABE,__user_mention__ __user_mention__ YASS THANKS ...,0
1,RT @AbnInfVet: ...And Then Bloomberg Says He D...,RT __user_mention__ ...And Then Bloomberg Says...,0
2,RT @lgbtqnation: Federal judge considers separ...,RT __user_mention__ Federal judge considers se...,0
3,RT @OKFosterWishes: URGENT; Beds needed to get...,RT __user_mention__ URGENT; Beds needed to get...,0
4,"RT @luke_brooks: N.America and EU! Our EP ""Wou...","RT __user_mention__ N.America and EU! Our EP ""...",0
...,...,...,...
248177,theawkwardmoment#theawkwardmoment when you sta...,theawkwardmoment#theawkwardmoment when you sta...,1
248178,I will have the rest of the site updated when ...,I will have the rest of the site updated when ...,1
248179,RT @EMANSANGELS: Follow @scottstorch and view ...,RT __user_mention__ Follow __user_mention__ an...,1
248180,Stupidity in numbers. penn state riot,Stupidity in numbers. penn state riot,1


In [5]:
n_samples = int(0.2 * len(df_tweets))
df_tweets_searches = resample(df_tweets, replace = False, n_samples = n_samples, random_state = 101)
df_tweets_searches = df_tweets_searches.reset_index(inplace = False, drop = True)
df_tweets_searches

,text,cleaned_text,isBot
0,No surprises there. It's still happening today...,No surprises there. It's still happening today...,0
1,I'm in waiting room and everybody staring at m...,I'm in waiting room and everybody staring at m...,0
2,RT @x_SupremeMee: Dead Serious . A Nigga Not F...,RT __user_mention__ Dead Serious . A Nigga Not...,0
3,Am I not destroying my enemies when I make fri...,Am I not destroying my enemies when I make fri...,1
4,RT @_christinareyes: today was such a good day...,RT __user_mention__ today was such a good day ...,0
...,...,...,...
49631,2 people followed me in the last day thanks to...,2 people followed me in the last day thanks to...,0
49632,@masonrudek congrats! We used to play Whitesbo...,__user_mention__ congrats! We used to play Whi...,0
49633,RT @sirenmoonbee: @AmaiaOrmazabal @ArtywoodSla...,RT __user_mention__ __user_mention__ __user_me...,0
49634,RT @NgaNgaDre: God is never too busy to answer...,RT __user_mention__ God is never too busy to a...,0


# Model
1. Logistic Regression (BoW)
2. Random Forest Classifier (BoW)
3. Logistic Regression (TF-IDF)
4. Random Forest Classifier (TF-IDF)
5. LSTM
6. Logistic Regression (BERT)
7. Random Forest Classifier (BERT)
8. Adaboost Classifier (BERT)
9. XGBoost (BERT)

## BERT Models (Tuning)

In [6]:
#initialising a pretrained bert model
tokenizer = BertTokenizer.from_pretrained("bert-base-cased", padding = True)
model = BertModel.from_pretrained("bert-base-cased")
nlp = pipeline("feature-extraction", tokenizer = tokenizer, model = model)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
#store all the tweets after they have been encoded by BERT

# feature_list = []

# for index, row in df_tweets_searches.iterrows():
#     #extracting the ith tweet and restricting the characters to 512, which is fine because twitter's limit is 280
#     text = row['cleaned_text'][:512]
#     #encoding all the individual words present in the tweet
#     vec = np.array(nlp(text))
#     #getting the mean representation of the words present in the tweet
#     vec = vec.reshape((vec.shape[1], vec.shape[2])).mean(axis = 0)
#     feature_list.append(vec)

# feature_vectors_searches = np.array(feature_list)

In [8]:
feature_vectors_searches = pd.read_csv("feature_vect_searches.csv")
feature_vectors_searches = feature_vectors_searches.drop(columns = "Unnamed: 0")

In [9]:
feature_vectors_searches

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.395461,0.314507,0.362490,0.213974,0.333369,-0.068762,0.316718,-0.207122,-0.338681,-0.032407,...,0.140077,0.063303,-0.153159,-0.073506,-0.095945,0.121968,0.373144,0.090164,0.037796,-0.182941
1,0.170986,0.041638,0.140698,0.080176,0.374009,-0.405314,0.350578,0.124787,-0.046961,-0.294437,...,-0.312045,0.146012,-0.363045,-0.148090,-0.066873,0.031754,0.166132,0.207321,0.134877,-0.004206
2,0.132554,0.117530,0.106922,0.173113,-0.003577,-0.298844,0.077240,-0.041078,0.187893,-0.018072,...,-0.176072,-0.032313,-0.135647,-0.070155,-0.281850,-0.359407,-0.040657,0.204591,-0.038190,0.070626
3,0.134241,-0.084924,-0.033736,0.272944,0.246399,-0.238337,0.177381,0.031927,0.326889,-0.293079,...,-0.300695,0.275399,-0.206875,-0.071240,0.496624,0.269286,-0.323785,0.088258,0.093973,-0.190454
4,0.056847,0.283105,-0.044481,0.177009,0.168187,-0.330898,0.139501,-0.185742,-0.129203,-0.143451,...,0.029934,0.447064,-0.177487,-0.134292,-0.215164,-0.281553,-0.075762,0.044960,0.103758,0.579587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49631,0.411459,0.223123,-0.125041,-0.059729,0.252963,-0.074129,0.155126,-0.043803,-0.352230,0.002781,...,-0.129086,-0.064744,-0.054788,-0.346518,-0.051776,0.288245,0.222222,-0.180260,-0.023279,0.192807
49632,0.131463,0.375077,-0.201440,0.118565,-0.016395,-0.155379,0.324532,-0.089360,-0.287430,-0.000772,...,-0.123567,0.238414,-0.164729,-0.314040,-0.374253,-0.295125,-0.109618,0.354500,0.005237,0.747328
49633,-0.039582,0.803484,-0.659892,0.127410,-0.208920,-0.011133,-0.068938,-0.277169,0.172432,-0.039849,...,-0.289599,0.052794,-0.687014,-0.323827,-0.179736,-0.648473,-0.302837,-0.052786,0.379125,1.124434
49634,0.242117,0.199913,-0.092624,-0.042763,0.238782,-0.338223,0.171441,-0.148693,-0.117149,-0.057121,...,-0.066954,0.280092,-0.107219,-0.149524,-0.042727,-0.337717,0.164451,0.209090,0.196218,0.371416


In [10]:
x_train_searches, x_test_searches, y_train_searches, y_test_searches = train_test_split(feature_vectors_searches, df_tweets_searches['isBot'], test_size = 0.2)

# Hyperparameter Tuning
We shall perform hyperparameter tuning for the models that use BERT features as the features. The method we will use to perform this tuning is Bayesian Optimisation, which is more efficient in finding the best hyperparameters as compared to Random Search and Grid Search as those 2 methods treat each trial of hyperparameters independently while Bayesian optimization is an informed search in the hyperparameter space in order to optimize the objective function.

However, in the context of Random Forest, since most of the features are discrete, we used random search instead.

Then, the tuned models will be tested on an independent test set and the best performing model will be selected based on this test set.

(Note, the results are likely to be different each time the notebook is run)

## 07 Random Forest Classifier (BERT)
* Model (Random)
* Error Metrics

### Model (Random)

In [12]:
# Model
rf_search = RandomForestClassifier()

random_rf = {"n_estimators": [100, 200, 600, 800], 
             "max_features": ["auto", "sqrt", "log2"], 
             "max_depth": [10, 20, 40, 80], 
             "min_samples_split": [10, 40, 80, 100], 
             "bootstrap": [True, False]}

random_search = RandomizedSearchCV(estimator = rf_search,
                                   param_distributions = random_rf,
                                   scoring = 'f1',
                                   n_jobs = -1,
                                   cv = 5,
                                   n_iter = 3,
                                   random_state = 101,
                                   verbose = 2)

In [13]:
%%time

# Execute search
results_random_rf = random_search.fit(x_train_searches, np.ravel(y_train_searches))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
CPU times: total: 10min 50s
Wall time: 59min 30s


In [14]:
print('Best Score: %s' % results_random_rf.best_score_)
print('Best Hyperparameters: %s' % results_random_rf.best_params_)

Best Score: 0.6642065703622988
Best Hyperparameters: {'n_estimators': 200, 'min_samples_split': 10, 'max_features': 'auto', 'max_depth': 80, 'bootstrap': False}


In [15]:
%%time

# y_prediction for the best model
y_pred_rf_searches = results_random_rf.predict(x_test_searches)

CPU times: total: 1.41 s
Wall time: 1.41 s


In [16]:
# Metrics for tuned random forest
accuracy = metrics.accuracy_score(y_test_searches, y_pred_rf_searches)
print(f'Accuracy: {accuracy}')
lg_loss = metrics.log_loss(y_test_searches, y_pred_rf_searches)
print(f'Log Loss: {lg_loss}')
roc_auc = metrics.roc_auc_score(y_test_searches, y_pred_rf_searches)
print(f'ROC AUC: {roc_auc}')
f1_score = metrics.f1_score(y_test_searches, y_pred_rf_searches)
print(f'F1-score: {f1_score}')
precision = metrics.precision_score(y_test_searches, y_pred_rf_searches)
print(f'Precision: {precision}')
recall = metrics.recall_score(y_test_searches, y_pred_rf_searches)
print(f'Recall: {recall}')
report = metrics.classification_report(y_test_searches, y_pred_rf_searches)
print(f'Classification Report: \n {report}')

Accuracy: 0.822522159548751
Log Loss: 6.129892574761366
ROC AUC: 0.7611389197447823
F1-score: 0.685581727337616
Precision: 0.8602776533811016
Recall: 0.5698605754968852
Classification Report: 
               precision    recall  f1-score   support

           0       0.81      0.95      0.88      6557
           1       0.86      0.57      0.69      3371

    accuracy                           0.82      9928
   macro avg       0.84      0.76      0.78      9928
weighted avg       0.83      0.82      0.81      9928



## 08 Adaboost Classifier (BERT)
* Model (Random)
* Error Metrics

### Model (Random)

In [17]:
adaboost_search = AdaBoostClassifier()

random_adaboost = {"n_estimators": [50, 100, 200, 500, 800], 
                   "learning_rate": [0.1, 1.0, 1.1, 1.2]}

random_search = RandomizedSearchCV(estimator = adaboost_search,
                                   param_distributions = random_adaboost,
                                   scoring = 'f1',
                                   n_jobs = -1,
                                   cv = 5,
                                   n_iter = 3,
                                   random_state = 101,
                                   verbose = 2)

In [18]:
%%time

results_random_adaboost = random_search.fit(x_train_searches, y_train_searches)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
CPU times: total: 19min 54s
Wall time: 1h 34min 10s


In [19]:
print('Best Score: %s' % results_random_adaboost.best_score_)
print('Best Hyperparameters: %s' % results_random_adaboost.best_params_)

Best Score: 0.5871279432502629
Best Hyperparameters: {'n_estimators': 200, 'learning_rate': 1.0}


In [20]:
%%time

# y_prediction for the best model
y_pred_ada_searches = results_random_adaboost.predict(x_test_searches)

CPU times: total: 6.41 s
Wall time: 6.42 s


In [21]:
# Metrics
accuracy = metrics.accuracy_score(y_test_searches, y_pred_ada_searches)
print(f'Accuracy: {accuracy}')
lg_loss = metrics.log_loss(y_test_searches, y_pred_ada_searches)
print(f'Log Loss: {lg_loss}')
roc_auc = metrics.roc_auc_score(y_test_searches, y_pred_ada_searches)
print(f'ROC AUC: {roc_auc}')
f1_score = metrics.f1_score(y_test_searches, y_pred_ada_searches)
print(f'F1-score: {f1_score}')
precision = metrics.precision_score(y_test_searches, y_pred_ada_searches)
print(f'Precision: {precision}')
recall = metrics.recall_score(y_test_searches, y_pred_ada_searches)
print(f'Recall: {recall}')
report = metrics.classification_report(y_test_searches, y_pred_ada_searches)
print(f'Classification Report: \n {report}')

Accuracy: 0.75
Log Loss: 8.634771094612367
ROC AUC: 0.7007584766441483
F1-score: 0.5978613091380428
Precision: 0.658693323812924
Recall: 0.5473153366953426
Classification Report: 
               precision    recall  f1-score   support

           0       0.79      0.85      0.82      6557
           1       0.66      0.55      0.60      3371

    accuracy                           0.75      9928
   macro avg       0.72      0.70      0.71      9928
weighted avg       0.74      0.75      0.74      9928



## 09 XGBoost Classifier (BERT)
* Model (Random)
* Error Metrics

### Model (Random)

In [22]:
weight_train_searches = y_train_searches.value_counts()[0] / y_train_searches.value_counts()[1]

xgb_search = XGBClassifier(scale_pos_weight = weight_train_searches,
                           gamma = 0.1,
                           alpha = 0.5,
                           objective = "binary:logistic",
                           eval_metric = "logloss")

random_xgb = {"max_depth": [5, 10, 15, 20, 25], 
            "subsample": [0.6, 0.8, 1.0], 
            "eta": [0.01, 0.1, 0.5], 
            "colsample_bytree": [0.7, 0.8, 0.9, 1.0]}

random_search = RandomizedSearchCV(estimator = xgb_search,
                                   param_distributions = random_xgb,
                                   scoring = 'f1',
                                   n_jobs = -1,
                                   cv = 5,
                                   n_iter = 3,
                                   random_state = 101,
                                   verbose = 2)

In [23]:
%%time

results_random_xgb = random_search.fit(x_train_searches, y_train_searches)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
CPU times: total: 32min 38s
Wall time: 1h 23min 38s


In [24]:
print('Best Score: %s' % results_random_xgb.best_score_)
print('Best Hyperparameters: %s' % results_random_xgb.best_params_)

Best Score: 0.689243445497582
Best Hyperparameters: {'subsample': 0.8, 'max_depth': 15, 'eta': 0.5, 'colsample_bytree': 0.7}


In [25]:
%%time

# y_prediction for the best model
y_pred_xgb_searches = results_random_xgb.predict(x_test_searches)

CPU times: total: 1.16 s
Wall time: 253 ms


In [26]:
# Metrics
accuracy = metrics.accuracy_score(y_test_searches, y_pred_xgb_searches)
print(f'Accuracy: {accuracy}')
lg_loss = metrics.log_loss(y_test_searches, y_pred_xgb_searches)
print(f'Log Loss: {lg_loss}')
roc_auc = metrics.roc_auc_score(y_test_searches, y_pred_xgb_searches)
print(f'ROC AUC: {roc_auc}')
f1_score = metrics.f1_score(y_test_searches, y_pred_xgb_searches)
print(f'F1-score: {f1_score}')
precision = metrics.precision_score(y_test_searches, y_pred_xgb_searches)
print(f'Precision: {precision}')
recall = metrics.recall_score(y_test_searches, y_pred_xgb_searches)
print(f'Recall: {recall}')
report = metrics.classification_report(y_test_searches, y_pred_xgb_searches)
print(f'Classification Report: \n {report}')

Accuracy: 0.8037872683319903
Log Loss: 6.77701362687161
ROC AUC: 0.7700899313131448
F1-score: 0.6971393034825871
Precision: 0.7324403789611238
Recall: 0.6650845446455058
Classification Report: 
               precision    recall  f1-score   support

           0       0.84      0.88      0.85      6557
           1       0.73      0.67      0.70      3371

    accuracy                           0.80      9928
   macro avg       0.78      0.77      0.78      9928
weighted avg       0.80      0.80      0.80      9928

